In [1]:
import json
import textwrap
import os
import jinja2

def render_template(template_file, context):
    loader = jinja2.FileSystemLoader(searchpath="./jinja_templates")
    env = jinja2.Environment(loader=loader)
    template = env.get_template(template_file)
    rendered_text = template.render(context)
    return rendered_text

def lower_no_spaces(s):
    return s.lower().replace(' ','-')

# BIOS

bios_string = textwrap.dedent('''\
+++
date = '2025-01-24T21:30:10-05:00'
draft = false
title = 'Bios'
+++

''')

with open('bios.json') as f:
    bios_data = json.load(f)
    for name,bio in bios_data.items():
        bios_string += f'## {name}\n\n{bio}\n\n'
        
with open('content/bios.md', 'w') as f:
    f.write(bios_string)
    
# SCHEDULES

def title_location_time(obj,day_string):
    return textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    **{obj['location']}**  
    {obj['time']}  
    ''')
    
def keynote(obj,day_string):
    return textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    **{obj['location']}**  
    {obj['time']}  
    
    {bios_data[obj['speaker']]}
    ''')
    
def work(obj):
    
    work_string = '\n\n'
    if 'description' in obj:
        work_string += f'{obj['description']}  \n'
    else:
        work_string += f'***{obj['title']}*** by {obj['composer']}  \n'
    
    for performer in obj['performers']:    
        work_string += f'&emsp;&emsp;*{performer}*  \n'
        
    work_string += '\n'
    return work_string

def write_to_content(string,file_name):
    with open(f'content/{file_name}', 'w') as f:
        f.write(string)

def to_pdf(string,file_name):
    with open('temp.md','w') as f:
        f.write(string)
    os.system(f'pandoc -i temp.md -o {file_name}')
    os.system('rm temp.md')

performer_avoiders = {'???', 'fixed media', 'Workshop Participants'}

def add_person(set,perf):
    if perf not in performer_avoiders:
        person = perf.split(',')[0]
        set[person] = None

def concert_page(obj,day_string):
    
    performers = {}
    composers = {}
        
    concert_page = textwrap.dedent(f'''
    +++
    date = '2025-01-24T21:30:10-05:00'
    draft = false
    title = '{obj['title']}'
    +++
    
    {day_string}  
    **{obj['location']}**  
    {obj['time']}  
    ''')
    
    for w in obj['works']:
        for p in w['performers']:
            add_person(performers,p)
        concert_page += work(w)
        
    concert_page += f'---\n\n## Program Notes\n\n'
    
    for w in obj['works']:
        
        if 'description' in w:
            concert_page += f'### *{w["description"]}*\n\n'
        else:
            concert_page += f'### *{w["title"]}*\n\n'
        
        if 'bios' in w:
            if len(w['bios']) > 1:
                concert_page += 'by:  \n'
                for b in w['bios']:
                    concert_page += f'[{b}](/bios/#{lower_no_spaces(b)})  \n'
                    add_person(composers,b)
                concert_page += '\n\n'
            else:
                concert_page += f'by [{w["bios"][0]}](/bios/#{lower_no_spaces(w["bios"][0])})\n\n'
                add_person(composers,w['bios'][0])
        else:
            add_person(composers,w['composer'])
            concert_page += f'by [{w["composer"]}](/bios/#{lower_no_spaces(w["composer"])})\n\n'
        
        concert_page += w["program-note"] + '\n\n'
        
    for key in list(composers.keys()):
        performers.pop(key, None) 
    
    if len(performers) > 0:
        concert_page += f'---\n\n## Performer Bios\n\n'
        
        for p in performers.keys():
            concert_page += f'[{p}](/bios/#{lower_no_spaces(p)})  \n'
        
        concert_page += '\n'
    
    write_to_content(concert_page,lower_no_spaces(obj['title']) + '.md')

def concert(obj,day_string):
    
    concert_page(obj,day_string)
    
    concert_string = textwrap.dedent(f'''
    ---
    ## {obj['title']}  

    **{obj['location']}**  
    {obj['time']}  
    [go to program](/{lower_no_spaces(obj['title'])})
    
    ''')
    
    for w in obj['works']:
        concert_string += work(w)
    
    return concert_string

def paper(obj):
    return f'**{obj["title"]}**  \n&emsp;&emsp;*{obj['author']}*  \n&emsp;&emsp;{obj["time"]}\n\n'
    
def paper_session_page(obj,day_string):
    string = textwrap.dedent(f'''
    +++
    date = '2025-01-24T21:30:10-05:00'
    draft = false
    title = '{obj['title']}'
    +++
    
    {day_string}  
    **{obj['location']}**  
    {obj['time']}  
    
    ''')
    
    for p in obj['presenters']:
        string += f'## {p['title']}\n\n'
        string += f'[{p["author"]}](/bios/#{lower_no_spaces(p['author'])})  \n'
        string += f'{p['time']}\n\n'
        if 'paper-link' in p:
            string += f'[go to paper](/papers/{p['paper-link']}.pdf)\n\n'
        string += p['abstract'] + '\n\n'
    
    write_to_content(string,lower_no_spaces(obj['title']) + '.md')
        

def paper_session(obj,day_string):
    
    paper_session_page(obj,day_string)
    
    paper_string = textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    **{obj['location']}**  
    {obj['time']}  
    [go to abstracts](/{lower_no_spaces(obj['title'])}/)
    
    ''')
    
    for p in obj['presenters']:
        paper_string += paper(p)
        
    return paper_string

def workshop_page(obj,day_string):
    
    string = textwrap.dedent(f'''
    +++
    date = '2025-01-24T21:30:10-05:00'
    draft = false
    title = '{obj['title']}'
    +++
    
    {day_string}  
    {obj['time']}
    
    ''')
    
    for w in obj['workshops']:
        # string += f'## {w["title"]}\n\n'
        # if len(w['led_by']) > 1:
        #     string += 'Led by:  \n'
        #     for f in w['led_by']:
        #         string += f'- [{f}](/bios/#{lower_no_spaces(f)})  \n'
        # else:
        #     string += f'Led by [{w["led_by"][0]}](/bios/#{lower_no_spaces(w["led_by"][0])})  \n'
            
        # if 'with_support_from' in w:
        #     string += f'With support from:  \n'
        #     for s in w['with_support_from']:
        #         string += f'- [{s}](/bios/#{lower_no_spaces(s)})  \n'
        
        # string += f'\n{w["location"]}\n\n'
        string += render_template('workshop_base.jinja.md',w)
        string += '\n\n' + w['abstract'] + '\n\n'
    
    write_to_content(string,lower_no_spaces(obj['title']) + '.md')
    
def workshop(obj,day_string):
    
    workshop_page(obj,day_string)
    
    workshop_string = textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    {obj['time']}
    
    [go to workshop descriptions](/{lower_no_spaces(obj['title'])})
        
    *Both workshops are one hour and occur simultaneously.*
    
    ''')
    
    for workshop in obj['workshops']:
        workshop_string += render_template('workshop_base.jinja.md',workshop)
    
    return workshop_string
    
def installations(obj,day_string):
    installations_string = textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    {obj['time']}  
    
    see all [installations](/installations)
    
    ''')
    
    for installation in obj['installations']:
        installations_string += textwrap.dedent(f'''
        &emsp;&emsp;**{installation['title']}**  
        &emsp;&emsp;&emsp;&emsp;*by {installation['artist']}*  
        &emsp;&emsp;&emsp;&emsp;**{installation['location']}**
        ''')
        
    return installations_string
    
event_processors = {
    'coffee': title_location_time,
    'keynote': keynote,
    'paper': paper_session,
    'workshop': workshop,
    'concert': concert,
    'installations': installations,
    'other': title_location_time
}

full_schedule = ''

sound_check_header = textwrap.dedent(f'''
        +++
        date = '2025-01-24T21:30:10-05:00'
        draft = false
        title = 'Sound Check Schedule'
        +++
        
        ''')

sound_check_schedule = ''

def render_sound_check_string(concert_obj):
    
    sound_check_string = ''
    
    reversed_works = concert_obj['works'][::-1]
    
    for w in reversed_works:
        
        performers = {}
        
        sound_check_string += f'**{w['sound_check']}**  \n'
        sound_check_string += f'&emsp;&emsp;***{w['title']}***  \n'
        sound_check_string += f'&emsp;&emsp;by {w['composer']}  \n'
        
        for p in w['performers']:
            add_person(performers,p)
        
        if 'bios' in w.keys():
            for c in w['bios']:
                performers.pop(c,None)
        else:
            performers.pop(w['composer'],None)
            
        if 'sound_checkers' in w:
            for sc in w['sound_checkers']:
                add_person(performers,sc)
            
        if len(performers) > 0:
            sound_check_string += '&emsp;&emsp;**Performers:**  \n'
            for p in performers:
                sound_check_string += f'&emsp;&emsp;&emsp;&emsp;{p}  \n'

        sound_check_string += '\n'
        
    return sound_check_string

papers_remote_doc = textwrap.dedent(f'''
# SuperCollider Symposium 2025 Paper Sessions

**Who is presenting remotely?**

''')

with open('program-data.json') as f:
    data = json.load(f)
    
    installations_page = textwrap.dedent(f'''
    +++
    date = '2025-01-24T21:30:10-05:00'
    draft = false
    title = 'Installations'
    +++
    
    ''')
    
    for i,day in enumerate(data['days']):
        day_prefix = textwrap.dedent(f'''
        +++
        date = '2025-01-24T21:30:10-05:00'
        draft = false
        title = 'Day {i+1}'
        +++
        
        ''')
        
        day_string = f'**{day['day']}**\n\n'
        
        sound_check_schedule += f'---\n\n## {day['day']}\n\n'
        
        if 'installations' in day:
            for installation in day['installations']:
                installation['day'] = day['day']
                installation['linkable_name'] = lower_no_spaces(installation['artist'])
                rendered_installation = render_template('installation.jinja.md',installation)
                installations_page += rendered_installation
                day_string += '## Installations:\n\n'
                day_string += 'see all [installations](/installations)\n\n'
                day_string += f'---\n\n***{installation["title"]}***  \n'
                day_string += f'by [{installation["artist"]}](/bios/#{lower_no_spaces(installation['artist'])})  \n**{installation["location"]}**\n\n'
                
        for event in day['events']:
            day_string += event_processors[event['type']](event,day['day'])
            if event['type'] == 'concert':
                sound_check_schedule += '\n\n---\n\n'
                sound_check_schedule += f'\n\n## {event['title']}\n\n'
                sound_check_schedule += render_sound_check_string(event)
                
            if event['type'] == 'installations':
                for installation in event['installations']:
                    installation['day'] = day['day']
                    installation['linkable_name'] = lower_no_spaces(installation['artist'])
                    installations_page += render_template('installation.jinja.md',installation)
                    
            if event['type'] == 'paper':
                papers_remote_doc += f'\n\n## {event["title"]}\n\n'
                papers_remote_doc += f'{day['day']}  \n**{event['location']}**  \n{event["time"]}\n\n'
                for p in event['presenters']:
                    if p['remote']:
                        papers_remote_doc += f'- **{p["author"]} (remote)**\n'
                    else:
                        papers_remote_doc += f'- {p['author']} (in person)\n'
                papers_remote_doc += '\n'
            
        full_schedule += f'\n\n---\n\n# Day {i+1}\n\n' + day_string
            
        write_to_content(day_prefix + day_string,f'day-{i+1}.md')

full_schedule = textwrap.dedent(f'''
        +++
        date = '2025-01-24T21:30:10-05:00'
        draft = false
        title = 'Full Schedule'
        +++
        
        ''') + full_schedule

write_to_content(full_schedule,'full-schedule.md')

sound_check_schedule_handout = '''
---
geometry: margin=1in
---
# SuperCollider Symposium 2025 Sound Check Schedule

''' + sound_check_schedule

sound_check_schedule_handout = sound_check_schedule_handout.replace('&emsp;','')    

to_pdf(sound_check_schedule_handout,'sound-check-schedule.pdf')

sound_check_schedule_with_header = sound_check_header + sound_check_schedule

write_to_content(sound_check_schedule_with_header,'sound-check-schedule.md')
write_to_content(installations_page,'installations.md')

with open('remote-paper-presenters.md','w') as f:
    f.write(papers_remote_doc)

os.system('pandoc -i remote-paper-presenters.md -o remote-paper-presenters.pdf')
os.system('rm remote-paper-presenters.md')


        

0